In [ ]:
# File: large-twist.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
from frontend import App

# create an app
app = App.create("large-twist")

# create a large cylinder mesh
V, F = app.mesh.cylinder(r=0.9, min_x=-1.75, max_x=1.75, n=1410)
app.asset.add.tri("cylinder", V, F)

# create a scene
scene = app.scene.create()

# add cylinder with gradient coloring along axis
obj = scene.add("cylinder").cylinder_color([0, 0, 0], [1, 0, 0], [0, 1, 0])

# set material properties
(
    obj.param.set("bend", 500.0)
    .set("contact-gap", 1.75e-3)
    .set("young-mod", 1e4)
    .set("poiss-rat", 0.25)
    .set("density", 3.5)
)

# define animation parameters for twisting and scaling
angular_vel, move_delta, t_end = 180.0, 0.5, 16.66
scale = 1 - t_end * 0.015

# pin and animate left end
(
    obj.pin(obj.grab([-1, 0, 0]))
    .spin(axis=[1, 0, 0], angular_velocity=angular_vel)
    .scale(scale, t_start=0.0, t_end=t_end)
    .move_by([move_delta, 0, 0], t_start=0.0, t_end=t_end)
)

# pin and animate right end in opposite direction
(
    obj.pin(obj.grab([1, 0, 0]))
    .spin(axis=[-1, 0, 0], angular_velocity=angular_vel)
    .scale(scale, t_start=0.0, t_end=t_end)
    .move_by([-move_delta, 0, 0], t_start=0.0, t_end=t_end)
)

# compile the scene and report stats
scene = scene.build().report()

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters - enable auto-save, disable gravity, and increase sparse matrix capacity
(
    session.param.set("auto-save", 10)
    .set("dt", 1 / 60)
    .set("frames", 2000)
    .set("gravity", 0.0)
    .set("csrmat-max-nnz", 160000000)
    .set("keep-verts", 100)
)

# build this session
session = session.build()

In [ ]:
# start the simulation (this example takes a long time)
session.start()

In [ ]:
# this example takes a long time...
# in case you shutdown the server (or kernel) and still want to restart
# from where you have (auto) saved, do this. Do not call cells above.

from frontend import App  # noqa

# recover the session from auto-saved state
session = App.recover("large-twist")

# resume if not currently running
if not App.busy():
    session.resume()

# preview the current state
session.preview()

# stream the logs
session.stream()